# DNAmFitAgeGripF

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [18]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import numpy as np
import pyaging as pya

## Instantiate model class

In [19]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.DNAmFitAgeGripF)

class DNAmFitAgeGripF(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x
    
    def postprocess(self, x):
        return x



In [20]:
model = pya.models.DNAmFitAgeGripF()

## Define clock metadata

In [21]:
model.metadata["clock_name"] = 'dnamfitagegripf'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2023
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "McGreevy, Kristen M., et al. \"DNAmFitAge: biological age indicator incorporating physical fitness.\" Aging (Albany NY) 15.10 (2023): 3904."
model.metadata["doi"] = 'https://doi.org/10.18632/aging.204538'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download GitHub repository

In [22]:
github_url = "https://github.com/kristenmcgreevy/DNAmFitAge.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

#### Download from R package

In [23]:
%%writefile download.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))
library(jsonlite)

DNAmFitnessModels <- readRDS("DNAmFitAge/DNAmFitnessModelsandFitAge_Oct2022.rds")

AllCpGs <- DNAmFitnessModels$AllCpGs
write_json(AllCpGs, "AllCpGs.json")

MaleMedians <- DNAmFitnessModels$Male_Medians_All
write.csv(MaleMedians, "MaleMedians.csv")
FemaleMedians <- DNAmFitnessModels$Female_Medians_All
write.csv(FemaleMedians, "FemaleMedians.csv")

Gait_noAge_Females <- DNAmFitnessModels$Gait_noAge_Females
Gait_noAge_Males <- DNAmFitnessModels$Gait_noAge_Males
Grip_noAge_Females <- DNAmFitnessModels$Grip_noAge_Females
Grip_noAge_Males <- DNAmFitnessModels$Grip_noAge_Males
VO2maxModel <- DNAmFitnessModels$VO2maxModel
write.csv(Gait_noAge_Females, "Gait_noAge_Females.csv")
write.csv(Gait_noAge_Males, "Gait_noAge_Males.csv")
write.csv(Grip_noAge_Females, "Grip_noAge_Females.csv")
write.csv(Grip_noAge_Males, "Grip_noAge_Males.csv")
write.csv(VO2maxModel, "VO2maxModel.csv")

Writing download.r


In [24]:
os.system("Rscript download.r")

0

## Load features

#### From JSON file

In [25]:
df = pd.read_csv('Grip_noAge_Females.csv', index_col=0)
df['feature'] = df['term']
df['coefficient'] = df['estimate']
model.features = df['feature'][1:].tolist()
df.head()

,term,step,estimate,lambda,dev.ratio,feature,coefficient
1,(Intercept),1,53.820647,0.3,0.395663,(Intercept),53.820647
2,cg00094518,1,-2.345767,0.3,0.395663,cg00094518,-2.345767
3,cg01854776,1,-2.941467,0.3,0.395663,cg01854776,-2.941467
4,cg02001410,1,2.813212,0.3,0.395663,cg02001410,2.813212
5,cg02036261,1,-0.942776,0.3,0.395663,cg02036261,-0.942776


## Load weights into base model

In [27]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'].iloc[0]])

#### Linear model

In [28]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

#### From CSV file

In [29]:
reference_df = pd.read_csv('FemaleMedians.csv', index_col=0)
model.reference_values =  list(reference_df.loc[1, model.features])

## Load preprocess and postprocess objects

In [30]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [31]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [32]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'McGreevy, Kristen M., et al. "DNAmFitAge: biological age '
             'indicator incorporating physical fitness." Aging (Albany NY) '
             '15.10 (2023): 3904.',
 'clock_name': 'dnamfitagegripf',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.204538',
 'notes': 'Reference values is mean between male and female training medians',
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2023}
reference_values: [0.115253497078786, 0.968105961834816, 0.0760619502414265, 0.0812081375780079, 0.674801040131638, 0.935404314255487, 0.868111085215553, 0.102187464459387, 0.875784927493447, 0.0721027662094031, 0.0794203726419054, 0.823996698486295, 0.417058282950929, 0.121432191141366, 0.59993557322113, 0.26732361189343, 0.631057715233057, 0.857492364335729, 0.9209946908

## Basic test

In [33]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 43.5148],
        [ 46.6872],
        [114.4924],
        [-12.2853],
        [ 45.3849],
        [  9.3484],
        [ 25.7228],
        [ 44.3546],
        [ 36.3137],
        [ -6.2013]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [34]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [35]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: Grip_noAge_Females.csv
Deleted file: Grip_noAge_Males.csv
Deleted file: Gait_noAge_Females.csv
Deleted file: VO2maxModel.csv
Deleted file: AllCpGs.json
Deleted file: Gait_noAge_Males.csv
Deleted folder: DNAmFitAge
Deleted file: download.r
Deleted file: FemaleMedians.csv
Deleted file: MaleMedians.csv
